In [25]:
import os

item_map = {
    'Realism': [],
    'Rococo': []
}

item_to_idx = {
    'Realism': 0,
    'Rococo': 0
}

data_dir = '/projects/data/wikiart/'
counter = 0
for category in item_map.keys():
    fnames = os.listdir(data_dir + category)
    
    for fn in fnames:
        item_map[category].append({
            'idx': counter,
            'label': item_to_idx[category],
            'filename': category + '/' + fn,
        })
        counter += 1
    
for k in item_map:
    print(k, len(item_map[k]))

Realism 10733
Rococo 2089


In [26]:
import random
for category in item_map.keys():
    random.shuffle(item_map[category])

In [27]:
def write_lst(image_arr, base_dir, file_path):
    with open(file_path, 'w') as f:
        count = 0
        for img in image_arr:
            label = img['label']
            img_path = os.path.join(base_dir, img['filename'])
            new_line = '\t'.join([str(count), str(label), str(img_path)])
            new_line += '\n'
            f.write(new_line)
            count += 1

In [28]:
import sys
min_data_len = sys.maxsize

for category in item_map.keys():
    min(min_data_len, len(item_map[category]))

sample = (0, 8)
train = (0, int(min_data_len * 0.7))
validation = (int(min_data_len * 0.7), int(min_data_len * 0.85))
test = (int(min_data_len * 0.85), int(min_data_len * 1))

def split_dataset(from_idx, to_idx):
    result = []
    for category in item_map.keys():
        result = result + item_map[category][from_idx: to_idx]
    
    return result

In [29]:
# sample set is for developing model and debugging
# because debugging with large dataset takes a long time
sample_set = split_dataset(sample[0], sample[1])
write_lst(sample_set, '/projects/data/wikiart/', '/projects/data/wikiart/sample.lst')

train_set = split_dataset(train[0], train[1])
write_lst(train_set, '/projects/data/wikiart/', '/projects/data/wikiart/train.lst')

validation_set = split_dataset(validation[0], validation[1])
write_lst(validation_set, '/projects/data/wikiart/', '/projects/data/wikiart/validation.lst')

test_set = split_dataset(test[0], test[1])
write_lst(validation_set, '/projects/data/wikiart/', '/projects/data/wikiart/test.lst')
